<a href="https://colab.research.google.com/github/yoderj/MyUtilsR/blob/master/about_time_2_api_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# time report

In [ ]:
import pandas as pd
import numpy as np
import datetime
import altair as alt
from urllib.parse import quote

pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.set_option('colheader_justify', 'right')

# config
group_name = 'msoe.edu/sdl/about-time-too'
milestone = 'q3-sprint3'

report_endpoint = 'https://about-time-api.herokuapp.com/api/report/' + quote(group_name, safe='') + '?milestone=' + quote(milestone)

df = pd.read_json(report_endpoint, orient='split')

def extract_name(x):
    return x['user']['username']

def extract_repo(x):
    return x['git_repo']['path']

def extract_gitlab_link(x):
    return x['task']['gitlab_link']

def extract_task_name(x):
    return x['task']['name']

def extract_task_id(x):
    return x['task']['id']

def extract_task_status(x):
    return x['task']['status']

format_hours = lambda x: '0' if x == 0 else "{0:.2f}".format(x).rstrip('0').rstrip('.') + 'h'

df['date_spent'] = pd.to_datetime(df['date_spent'], dayfirst=True)
df['date_spent'] = df['date_spent']
df['user'] = df.apply(extract_name, axis=1)
df['git_repo'] = df.apply(extract_repo, axis=1)
df['link'] = df.apply(extract_gitlab_link, axis=1)
df['task_name'] = df.apply(extract_task_name, axis=1)
df['task_status'] = df.apply(extract_task_status, axis=1)
del df['task']
del df['git_repo']
del df['id']


# deal with time corrections
df = df.groupby(['date_spent', 'user', 'link', 'task_name', 'task_status'], as_index=False).agg('sum')
df = df[0 != df.hours_spent]

df['year'] = df['date_spent'].dt.strftime('%Y')
df['month'] = df['date_spent'].dt.strftime('%m')
df['day'] = df['date_spent'].dt.strftime('%d (%a)')
df['week'] = df['date_spent'].dt.strftime('%W')
df['day_of_week'] = df['date_spent'].dt.strftime('%a')

df.sort_values(by=['date_spent', 'user'], inplace=True, ascending=False)
df = df.reset_index(drop=True)

## hours per day

## hours per week

In [ ]:
res = df.pivot_table(
    index='user',
    columns=['year', 'week', 'month', 'day'],
    values='hours_spent',
    aggfunc=[np.sum],
    fill_value=0,
    margins=True,
    margins_name='Σ'
)
res.style.format(format_hours)

In [ ]:
res_weeks = df.pivot_table(
    index='user',
    columns=['week'],
    values='hours_spent',
    aggfunc=[np.sum],
    fill_value=0,
    margins=True,
    margins_name='Σ'
)
res_weeks.style.format(format_hours)

## hours per day visual

In [ ]:
chart_data = df.groupby(['user', 'date_spent'], as_index=False).agg('sum')

alt.Chart(chart_data).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('date_spent:O', axis=alt.Axis(labelAngle=0)),
    alt.Y('user:N'),
    alt.Size('hours_spent:Q',
        scale=alt.Scale(range=[0, 5000]),
        legend=alt.Legend(title='hours logged in day')
    ),
    alt.Color('user:N', legend=None)
).properties(
    width=900,
    height=500
)

Chart({
  data:             user date_spent  hours_spent
  0       LilyJune 2019-05-02         2.00
  1       LilyJune 2019-05-07         4.50
  2       LilyJune 2019-05-08         3.00
  3       LilyJune 2019-05-09         2.00
  4       LilyJune 2019-05-12         6.00
  5       LilyJune 2019-05-14         4.00
  6       LilyJune 2019-05-16         5.00
  7       LilyJune 2019-05-18         1.50
  8       LilyJune 2019-05-19         3.50
  9        gieseba 2019-05-02         2.00
  10       gieseba 2019-05-05         1.00
  11       gieseba 2019-05-07         3.00
  12       gieseba 2019-05-09         2.00
  13       gieseba 2019-05-10         1.00
  14       gieseba 2019-05-12         4.00
  15       gieseba 2019-05-14         2.00
  16       gieseba 2019-05-16         3.00
  17       gieseba 2019-05-19         2.25
  18       gieseba 2019-05-20         6.00
  19     martensdr 2019-05-02         2.00
  20     martensdr 2019-05-07         3.00
  21     martensdr 2019-05-08         2.50
  22     martensdr 2019-05-09         2.00
  23     martensdr 2019-05-10         2.00
  24     martensdr 2019-05-14         2.00
  25     martensdr 2019-05-15         4.00
  26     martensdr 2019-05-16         3.00
  27     martensdr 2019-05-19         4.50
  28     martensdr 2019-05-20         3.00
  29  noe.gonzalez 2019-05-02         2.00
  30  noe.gonzalez 2019-05-05         2.00
  31  noe.gonzalez 2019-05-06         2.00
  32  noe.gonzalez 2019-05-07         2.00
  33  noe.gonzalez 2019-05-08         1.00
  34  noe.gonzalez 2019-05-09         2.00
  35  noe.gonzalez 2019-05-10         2.00
  36  noe.gonzalez 2019-05-12         3.00
  37  noe.gonzalez 2019-05-14         2.00
  38  noe.gonzalez 2019-05-16         2.00
  39  noe.gonzalez 2019-05-18         6.00
  40  noe.gonzalez 2019-05-19         1.00
  41  noe.gonzalez 2019-05-20         2.00
  42         slang 2019-05-02         2.00
  43         slang 2019-05-09         2.00
  44         slang 2019-05-14         4.00
  45         slang 2019-05-16         3.00
  46         slang 2019-05-18         4.00
  47         slang 2019-05-19         9.50
  48         slang 2019-05-20         5.50
  49    stanglersm 2019-05-02         2.00
  50    stanglersm 2019-05-07         4.00
  51    stanglersm 2019-05-08         2.00
  52    stanglersm 2019-05-09         2.00
  53    stanglersm 2019-05-13         1.00
  54    stanglersm 2019-05-14         2.50
  55    stanglersm 2019-05-15         2.50
  56    stanglersm 2019-05-16         2.00
  57    stanglersm 2019-05-19        11.00
  58    stanglersm 2019-05-20         1.50,
  encoding: FacetedEncoding({
    color: Color({
      legend: None,
      shorthand: 'user:N'
    }),
    size: Size({
      legend: Legend({
        title: 'hours logged in day'
      }),
      scale: Scale({
        range: [0, 5000]
      }),
      shorthand: 'hours_spent:Q'
    }),
    x: X({
      axis: Axis({
        labelAngle: 0
      }),
      shorthand: 'date_spent:O'
    }),
    y: Y({
      shorthand: 'user:N'
    })
  }),
  height: 500,
  mark: MarkDef({
    opacity: 0.8,
    stroke: 'black',
    strokeWidth: 1,
    type: 'circle'
  }),
  width: 900
})

## hours by day of week

In [ ]:
chart_data = df.groupby(['user', 'day_of_week'], as_index=False).agg('sum')

alt.Chart(chart_data).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    alt.X('day_of_week:O', axis=alt.Axis(labelAngle=0), sort=['Mon','Tue','Wed','Thu','Fri','Sat', 'Sun']),
    alt.Y('user:N'),
    alt.Size('hours_spent:Q',
        scale=alt.Scale(range=[0, 5000]),
        legend=alt.Legend(title='hours logged in day of week')
    ),
    alt.Color('user:N', legend=None)
).properties(
    width=900,
    height=500
)

Chart({
  data:             user day_of_week  hours_spent
  0       LilyJune         Sat         1.50
  1       LilyJune         Sun         9.50
  2       LilyJune         Thu         9.00
  3       LilyJune         Tue         8.50
  4       LilyJune         Wed         3.00
  5        gieseba         Fri         1.00
  6        gieseba         Mon         6.00
  7        gieseba         Sun         7.25
  8        gieseba         Thu         7.00
  9        gieseba         Tue         5.00
  10     martensdr         Fri         2.00
  11     martensdr         Mon         3.00
  12     martensdr         Sun         4.50
  13     martensdr         Thu         7.00
  14     martensdr         Tue         5.00
  15     martensdr         Wed         6.50
  16  noe.gonzalez         Fri         2.00
  17  noe.gonzalez         Mon         4.00
  18  noe.gonzalez         Sat         6.00
  19  noe.gonzalez         Sun         6.00
  20  noe.gonzalez         Thu         6.00
  21  noe.gonzalez         Tue         4.00
  22  noe.gonzalez         Wed         1.00
  23         slang         Mon         5.50
  24         slang         Sat         4.00
  25         slang         Sun         9.50
  26         slang         Thu         7.00
  27         slang         Tue         4.00
  28    stanglersm         Mon         2.50
  29    stanglersm         Sun        11.00
  30    stanglersm         Thu         6.00
  31    stanglersm         Tue         6.50
  32    stanglersm         Wed         4.50,
  encoding: FacetedEncoding({
    color: Color({
      legend: None,
      shorthand: 'user:N'
    }),
    size: Size({
      legend: Legend({
        title: 'hours logged in day of week'
      }),
      scale: Scale({
        range: [0, 5000]
      }),
      shorthand: 'hours_spent:Q'
    }),
    x: X({
      axis: Axis({
        labelAngle: 0
      }),
      shorthand: 'day_of_week:O',
      sort: ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    }),
    y: Y({
      shorthand: 'user:N'
    })
  }),
  height: 500,
  mark: MarkDef({
    opacity: 0.8,
    stroke: 'black',
    strokeWidth: 1,
    type: 'circle'
  }),
  width: 900
})

## hours per issue

In [ ]:
# regroup because some PBI mappings group issues/MRs 
chart_data = df.groupby(['user', 'task_name'], as_index=False).agg('sum')

base = alt.Chart(chart_data)
scale = alt.Scale(paddingInner=0)
heatmap = base.mark_rect().encode(
    alt.X('user:O', scale=scale),
    alt.Y('task_name:O', scale=scale),
    color='hours_spent'
)

text = base.mark_text(baseline='middle').encode(
    x='user:O',
    y='task_name:O',
    text='hours_spent',
    color=alt.condition(
        alt.datum['hours_spent'] < 7,
        alt.value('black'),
        alt.value('white')
    )
)

heatmap + text

LayerChart({
  layer: [Chart({
    data:             user                                          task_name  \
    0       LilyJune                                              Clock   
    1       LilyJune         Deploy to Netlify & Setup Auto Deployments   
    2       LilyJune                                            Favicon   
    3       LilyJune  Handle view with no dates selected & refactor ...   
    4       LilyJune  Parse markdown from descriptions into all data...   
    5       LilyJune                             Q3 Sprint 3 Ceremonies   
    6       LilyJune                          Resolve "Filter by Group"   
    7       LilyJune       Resolve "Gitlab scrum doc Table of Contents"   
    8       LilyJune  Resolve "Parse markdown from descriptions into...   
    9       LilyJune                                          Tutorials   
    10       gieseba                                              Clock   
    11       gieseba                                    Filter by Group   
    12       gieseba                 Gitlab scrum doc Table of Contents   
    13       gieseba  Link relevant blog posts about using Gitlab to...   
    14       gieseba                             Q3 Sprint 3 Ceremonies   
    15     martensdr  Establish workflow for using gitlab and includ...   
    16     martensdr                                            Favicon   
    17     martensdr                                 Front page styling   
    18     martensdr                 Gitlab scrum doc Table of Contents   
    19     martensdr                             Q3 Sprint 3 Ceremonies   
    20  noe.gonzalez  Bug: Descriptions that are too long cause the ...   
    21  noe.gonzalez                  GET STARTED: A guide for students   
    22  noe.gonzalez                             Q3 Sprint 3 Ceremonies   
    23  noe.gonzalez                          add group report endpoint   
    24         slang                                        Add a clock   
    25         slang                 Gitlab scrum doc Table of Contents   
    26         slang                           Link to Jupyter Notebook   
    27         slang                Opening repo link in new tab breaks   
    28         slang                             Q3 Sprint 3 Ceremonies   
    29         slang             Report filtering by date and milestone   
    30         slang                          add group report endpoint   
    31         slang                     add group_id to milestone view   
    32         slang            add query params to milestones endpoint   
    33         slang  fix issue with too many requests being sent to...   
    34         slang            fix missing key attribute on repo links   
    35         slang  return no entries when an incorrect group path...   
    36         slang                scrape all groups for a given token   
    37    stanglersm  Establish workflow for using gitlab and includ...   
    38    stanglersm                                Filter by Milestone   
    39    stanglersm                             Q3 Sprint 3 Ceremonies   
    40    stanglersm                             Select Project to View   
    41    stanglersm                                  TEST LOGGING TIME   
    
        hours_spent  
    0          1.00  
    1          1.50  
    2          2.00  
    3          9.00  
    4          2.00  
    5          2.00  
    6          1.50  
    7          1.50  
    8          1.50  
    9          9.50  
    10         1.00  
    11         0.25  
    12        14.00  
    13         1.00  
    14        10.00  
    15         4.00  
    16         0.50  
    17         2.00  
    18        15.50  
    19         6.00  
    20         6.00  
    21         2.00  
    22         5.00  
    23        16.00  
    24         2.00  
    25         2.00  
    26         4.00  
    27         1.00  
    28         4.50  
    29         3.50  
    30         2.00  
    31         1.00  
    32 

## full time logs

In [ ]:
from numbers import Number

def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if isinstance(val, Number) and val < 0 else 'black'
    return 'color: %s' % color

df.style.format({
    'hours_spent': format_hours
}).hide_columns([
    'day',
    'week',
    'month',
    'year',
    'day_of_week'
]).applymap(color_negative_red)

,date_spent,user,link,task_name,task_status,hours_spent
0,2019-05-20 00:00:00,stanglersm,https://gitlab.com/msoe.edu/sdl/about-time-too/testing-time-logs/issues/1,TEST LOGGING TIME,opened,1.5h
1,2019-05-20 00:00:00,slang,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/issues/42,Q3 Sprint 3 Ceremonies,opened,1.5h
2,2019-05-20 00:00:00,slang,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/merge_requests/29,fix missing key attribute on repo links,merged,1h
3,2019-05-20 00:00:00,slang,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/merge_requests/30,Add a clock,merged,2h
4,2019-05-20 00:00:00,slang,https://gitlab.com/msoe.edu/sdl/about-time-too/scraper/merge_requests/17,fix issue with too many requests being sent to gitlab api,merged,1h
5,2019-05-20 00:00:00,noe.gonzalez,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/issues/42,Q3 Sprint 3 Ceremonies,opened,2h
6,2019-05-20 00:00:00,martensdr,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/issues/42,Q3 Sprint 3 Ceremonies,opened,3h
7,2019-05-20 00:00:00,gieseba,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/issues/42,Q3 Sprint 3 Ceremonies,opened,6h
8,2019-05-19 00:00:00,stanglersm,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/issues/19,Filter by Milestone,closed,10h
9,2019-05-19 00:00:00,stanglersm,https://gitlab.com/msoe.edu/sdl/about-time-too/frontend/issues/42,Q3 Sprint 3 Ceremonies,opened,1h


In [ ]:
# show number of days in each numbered week (sanity check)
df.groupby(['week'])['day'].nunique()

week
17    2
18    6
19    6
20    1
Name: day, dtype: int64